# Installiert die Google OR Tools

Notwendig, wenn das Notebook via Google Colab Research gestartet wird.
Falls das Notebook lokal aufgerufen wird, muss der Befehl 

'python pip install ortools'

in der Console aufgerufen werden

In [ ]:
!pip install ortools

# Solver initialisieren

In [ ]:
from ortools.linear_solver import pywraplp
 

# Initialisiere und definiere den Solver
solver = pywraplp.Solver('SLULSP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)



# Parameterdeklaration

Lädt die Parameter

In [ ]:
# PARAMETERDEFINITION
d = [20,10,30,10]
l = 1
s = 20
M = sum(d)
T = len(d)

# Variablendeklaration

Legt die Variablen an und definiert den Wertebereich

In [ ]:
# VARIABLENDEKLARATION 
L = [[]]*T
X = [[]]*T
y = [[]]*T 
    
for t in range(T): 
  L[t] = solver.NumVar(0.0, solver.infinity(),  "L_%s"%(t))
  X[t] = solver.NumVar(0.0, solver.infinity(),"")
  y[t] = solver.BoolVar("")

# Nebenbedingungen

Legt die Lagerbilanzgleichung und die Rüstrestriktion

In [ ]:
#### DEFINITION DER NEBENBEDINGUNGEN ####

# (2) Lagerbilanzgleichung

# Erste Periode (ohne Anfangslagerbestand)
solver.Add(X[0] - d[0] == L[0])
# Alle anderen Perioden 
# (unter Berücksichtigung des Lagerendbestands der Vorperiode)
for t in range(1,T):
  solver.Add(L[t-1]+X[t]-d[t]==L[t])
 
# (3) Rüstrestriktion
for t in range(T):
  solver.Add(X[t] <= y[t]*M)

# Definition der Zielfunktion 
Minimerung der Summe aus Lage- und Rüstkosten

In [ ]:
# DEFINITION DER ZIELFUNKTION
objective = solver.Objective()
for t in range(T):
  objective.SetCoefficient(L[t], l) # Multipliziert L[t] mit l und fügt ein "+" hinter
  objective.SetCoefficient(y[t], s) # Multipliziert y[t] mit s und fügt ein "+" hinter
objective.SetMinimization()

# Triggern des Lösungsvorgangs

Ausgabe der Lösung

In [ ]:
# TRIGGERN DES LÖSUNGSVORGANGS
status = solver.Solve()
    
 
# AUSGABE DER LÖSUNG
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

if status == pywraplp.Solver.OPTIMAL:
  print('Lösung:')
  print('Zielfunktionswert =', solver.Objective().Value())
  print('X =', [X[t].solution_value() for t in range(T)])
  print('y =', [y[t].solution_value() for t in range(T)])
  print("L =", [L[t].solution_value() for t in range(T)])
else:
  print('The problem does not have an optimal solution.')
    
# AUSGABE DER LÖSUNGSZEIT UND ANGABEN ZUM LÖSUNGSVORGANG
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    
